# Mineral Resource estimation with Python

Welcome to the **world of Python**. 

This is a long exercise. Here you will:
 - create drillholes, and a geological model
 - tag drillholes with domain
 - create a block model and calculate percentage of mineralized material
 - composite
 - do statistical analysis (variography not implemented but explained)
 - interpolate grade and validate interpolations
 - Report resources
 
 All this using Python and Paraview. The figure below shows some of the outcomes of this exercise
 
 <img src = 'figures/fig1.JPG'>
 
 Before we start let's do a quick review of Jupyter (this IDE) and python
 
 

## Exercise 1 Introduction to Jupyter and Python

Familiarizing yourself with Jupyter:
 - Open jupyter notebook (you are already in)
 - Rename the file as "my MRE using python" (make a copy to keep the original file unchanged)
 - Add Markdown cell and type some comments (double click here! to see the markdown)

Python introduction
 - Import libraries  
 - Import drillhole data located in folder `data/` into pandas DataFrames
 - Explore the data using pandas and plot collar location 
 - Create a column of $log_{e}(Au)$ in table assays 

Note: **Double click here to see the code of the cell. ** This will give you an idea about writing comments in markdown


In [ ]:
# these are magic commands used to control Jupyter's behavior 
%gui qt                            
%matplotlib inline 
#%matplotlib notebook

In [ ]:
# Import libraries 
import pandas as pd               # this imports pandas as pd
import pygslib
import numpy as np
import matplotlib.pyplot as plt         

Some tips:
 - use key `TAB` to do autocompletion. Example type `pd.rea` and press `TAB`
 - use keys `CHIFT+TAB` to see help  
 - use type `?<python object>` and `Enter` to print help, also `help(<python object>)`, and also `print(<python object>.__doc__)`

In [ ]:
# Import drillhole data located in folder data/ into pandas dataframes
collar = pd.read_csv('data/collar.csv')
survey = pd.read_csv('data/survey.csv')
assay  = pd.read_csv('data/assay.csv')

Explore the data using pandas and plot collar location

In [ ]:
# plot first few lines
collar.head()

In [ ]:
# see column types and names
print ('*** Table Collar ***\n', collar.dtypes)
print ('*** Table Survey ***\n', survey.dtypes)
print ('*** Interval Table ***\n', assay.dtypes)

<div class="alert alert-danger">
<b>Important!</b> In PyGSLIB column names and types are prescribed, like in Datamine. Each table type is supposed to have certain "system" columns, as in this example. Names are case-sensitive, and "system" compulsory names are usually in upper case.  
</div>

In [ ]:
# summary stats for Au 
assay['Au'].describe()

In [ ]:
# calculate Au log
assay['log(Au)'] = np.log(assay['Au']+0.01)

In [ ]:
# quick histogram of Au and Au log
assay[['Au', 'log(Au)']].hist()

In [ ]:
# plot collars 
collar.plot.scatter(x='XCOLLAR', y= 'YCOLLAR')

### Paraview
Plot collars in Paraview: 
- drag and drop collar table into Paraview
- use filter `table to points`
- use gaussian points visualization

![figures/fig2.JPG](figures/fig2.JPG)

## Exercise 2: Drillholes 

Exploratory data analysis
 - Create a drillhole object
 - Add drillhole intervals
 - Validate drillhole object and interval table
 - Desurvey 
 - Export drillhole as vtk file and identify mineralization (tip use `tube filter`)


In [ ]:
# create drillhole object
mydholes = pygslib.drillhole.Drillhole(collar, survey)   # note that Drillhole is a class and mydholes is an instance of this class

# add interval table
mydholes.addtable(table = assay, table_name = 'assay')  # here we are using named parameters, the order is not important

In [ ]:
# validate
mydholes.validate()
mydholes.validate_table('assay')

if no red box then the drillhole is ok, you are good to go

In [ ]:
# tables are stored in a list of panda arrays named table, within drillhole object
mydholes.table['assay'].head()

In [ ]:
mydholes.desurvey('assay')     # desurvey 
mydholes.table['assay'].head() # see what happens

 - xm,ym,zm are the coordinates of the middle interval
 
you also have endpoints

- xb,yb,zb are the coordinates of the beginning of the interval 
- xe,ye,ze are the coordinates of the end of the interval

Endpoints are optional but are required to generate vtk files. To disable endpoints use `endpoints=False` in function `desurvey()`

inclinations of intervals, azm, and dip, are also calculated.  

In [ ]:
# Export drillhole as vtk file and identify mineralization (tip use tube filter)
mydholes.intervals2vtk(table_name= 'assay', filename= 'assay')

Load the file in Paraview (drag and drop). Use log scale to visualize Au grades, with interval from 0.01 to 5 g/t. It may look like this:  

<img src = 'figures/fig3.JPG' width = '50%'>

## Exercise 3: 

The mineralization looks like a simple horizon. It makes sense modeling this as surfaces. Because it is so simple, we can use queries from pandas to extract contact points. 

Modeling
 - Label drillhole intervals with domain (tip use key composite to simplify)
 - Extract contact points to model surfaces (tip can use queries in pandas or manual selection in Paraview)
 - Create contact surfaces and topo interpolating with Rbf (vtkPolyData with open surface triangulations)
 - Define working region extent

Create solids
 - Convert vtkPolyData surfaces to implicit functions  
 - Model solids using:
     - cutting tool or 
     - evaluating distances

Tag drillhole data
 - Tag drillhole data with domain (optional in this case)
 - composite drillholes


In [ ]:
# a) create domain using pandas filters (see help on command .loc for more info)
mydholes.table['assay']['CMPDOM'] = 0   # new field all =  0 
mydholes.table['assay'].loc[mydholes.table['assay']['Au']>0.01,'CMPDOM'] = 1 # we select all intervals with Au> 0.01 and set CMPDOM = 1

mydholes.table['assay'][['BHID', 'FROM', 'TO', 'Au', 'CMPDOM']]

In [ ]:
# b) composite by domain  
mydholes.key_composite(table_name='assay', key_name='CMPDOM', variable_name= 'Au', new_table_name = 'litho',overwrite = True)
mydholes.table['litho'].head()

In [ ]:
# compositing removes coordinates, you need to desurvey again
mydholes.desurvey('litho')

In [ ]:
# now we can get coordinates of hanging and footwall
hw = mydholes.table['litho'].loc[mydholes.table['litho']['CMPDOM']==1, ['xb','yb','zb']]
fw = mydholes.table['litho'].loc[mydholes.table['litho']['CMPDOM']==1, ['xe','ye','ze']]

# save the points to review it in Paraview
hw.to_csv('hw.csv', index= False)
fw.to_csv('fw.csv', index= False)

You may see something like this in Paraview

<img src = 'figures/fig4.JPG' width = '50%'>

### Defining working region and modeling surfaces

Surfaces and solids are generated within a working region. 

In [ ]:
# define working region extent and point spacing
xorg = -10
yorg = -10
zorg = -10
dx = 5
dy = 5
dz = 5
nx = 40
ny = 44
nz = 36

In [ ]:
# generate vtk open surfaces
# the output is a mesh in vtk format, and node coordinates
topo_pd,x_topo,y_topo,z_topo = pygslib.vtktools.rbfinterpolate( x=mydholes.collar['XCOLLAR'].values,
                                                                y=mydholes.collar['YCOLLAR'].values,
                                                                z=mydholes.collar['ZCOLLAR'].values,
                                                                xorg=xorg, yorg=yorg,dx=dx,dy=dy,nx=nx,ny=ny)

hw_pd,x_hw,y_hw,z_hw  = pygslib.vtktools.rbfinterpolate(  x=hw['xb'].values,
                                                       y=hw['yb'].values,
                                                       z=hw['zb'].values,
                                                       xorg=xorg, yorg=yorg,dx=dx,dy=dy,nx=nx,ny=ny)

fw_pd,x_hw,y_hw,z_hw  = pygslib.vtktools.rbfinterpolate(  x=fw['xe'].values,
                                                       y=fw['ye'].values,
                                                       z=fw['ze'].values,
                                                       xorg=xorg, yorg=yorg,dx=dx,dy=dy,nx=nx,ny=ny)


# and save the surfaces
pygslib.vtktools.SavePolydata(topo_pd, 'topo')
pygslib.vtktools.SavePolydata(hw_pd, 'hw')
pygslib.vtktools.SavePolydata(fw_pd, 'fw')

<img src = 'figures/fig5.JPG' width = '50%'>

Now we need a region,

A region is a vtk grid with regularly spacing points and optionally contact points (snapping). The grid has octahedron cells generated with Delaunay 3D triangulation. This takes a bit of time, be patient... 

In [ ]:
# this is a grid (a box, we cut to generate geology). We can generate a grid or tetras with surface point included to emulate snapping 
region = pygslib.vtktools.define_region_grid(xorg, yorg, zorg, dx, dy,  dz, nx, ny, nz, snapping_points = [topo_pd,hw_pd,fw_pd])
pygslib.vtktools.SaveUnstructuredGrid(region, "region")

### Create solids
Solids will be created cutting or evaluating distances. Both involve implicit functions and require implicit surfaces


In [ ]:
# Convert vtkPolyData surfaces to implicit functions
impl_topo = pygslib.vtktools.implicit_surface(topo_pd)
impl_hw = pygslib.vtktools.implicit_surface(hw_pd)
impl_fw = pygslib.vtktools.implicit_surface(fw_pd)

Now we can generate solids

Option (a) using cutter (clip), we basically slice a region with implicit surfaces. This is not very stable, we prefer option b)

In [ ]:
# get model below topo
#topo_region_c,topo_solid_c = pygslib.vtktools.clip_with_surface(region, implicit_surface = impl_topo, how='outside')
#topo_region_c,topo_solid_c = pygslib.vtktools.clip_with_surface(topo_region_c, implicit_surface = impl_hw, how='inside')
#pygslib.vtktools.SavePolydata(topo_solid_c, 'topo_solid_c')

# get model between hw and fw
#d1_region_c,d1_solid_c = pygslib.vtktools.clip_with_surface(region, implicit_surface = impl_hw, how='outside')
#d1_region_c,d1_solid_c = pygslib.vtktools.clip_with_surface(d1_region_c, implicit_surface = impl_fw, how='inside')
#pygslib.vtktools.SavePolydata(d1_solid_c, 'd1_solid_c')

# get model below  fw
#base_region_c,base_solid_c = pygslib.vtktools.clip_with_surface(region, implicit_surface = impl_fw, how='outside')
#pygslib.vtktools.SavePolydata(base_solid_c , 'base_solid_c')

Option (b) evaluating a region with implicit surfaces

In [ ]:
# evaluate surfaces
#below topo
region,topo_d = pygslib.vtktools.evaluate_region(region, implicit_func = impl_topo, func_name='topo_d', invert=False, capt = -10000)
#above hanging wall
region, hw_u = pygslib.vtktools.evaluate_region(region, implicit_func = impl_hw, func_name='hw_u', invert=True, capt = -10000)
#below hanging wall
region, hw_d = pygslib.vtktools.evaluate_region(region, implicit_func = impl_hw, func_name='hw_d', invert=False, capt = -10000)
#above footwall
region, fw_u = pygslib.vtktools.evaluate_region(region, implicit_func = impl_fw, func_name='fw_u', invert=True, capt = -10000)
#below footwall
region, fw_d = pygslib.vtktools.evaluate_region(region, implicit_func = impl_fw, func_name='fw_d', invert=False, capt = -10000)

now we can use regions to:
 - do boolean operations 
 - extract surfaces

In [ ]:
# create intersection between hanging wall and foot wall
dom1= np.minimum(hw_d, fw_u)
region = pygslib.vtktools.set_region_field(region, dom1, 'dom1')
# extract surface
dom1_poly = pygslib.vtktools.extract_surface(region,'dom1')
# Save surface
pygslib.vtktools.SavePolydata(dom1_poly, 'dom1')

In [ ]:
# create intersection between topo and hanging wall
dom_topo= np.minimum(topo_d, hw_u)
region = pygslib.vtktools.set_region_field(region, dom_topo, 'dom_topo')
# extract surface
dom_topo_poly = pygslib.vtktools.extract_surface(region,'dom_topo')
# Save surface
pygslib.vtktools.SavePolydata(dom_topo_poly, 'dom_topo')

In [ ]:
# not boolean required below fw
# extract surface
dom_fw_poly = pygslib.vtktools.extract_surface(region,'fw_d')
# Save surface
pygslib.vtktools.SavePolydata(dom_fw_poly, 'dom_fw')

### Drillholes tagging

Tagging assigns a code to drillholes within a domain, usually defined by wireframes. 

We already have drillhole tags but we show you how to do it. You will need to do this if wireframes are created manually or imported. The functions that can tag are:

 - `pygslib.vtktools.pointinsolid()` This is the preferred way, requires closed solids. 
 - `pygslib.vtktools.evaluate_implicit_points()`. New and experimental, uses implicit functions
 - `pygslib.vtktools.pointquering()`. High level function.
 - `pygslib.vtktools.vtk_raycasting()`. Low level function.  Used by pointinsolid()



In [ ]:
# creating array to tag samples in domain1
inside1=pygslib.vtktools.pointinsolid(dom1_poly, 
                       x=mydholes.table['assay']['xm'].values, # .values this extracts numpy array from pandas
                       y=mydholes.table['assay']['ym'].values, 
                       z=mydholes.table['assay']['zm'].values)

# creating a new domain field 
mydholes.table['assay']['Domain1']=inside1.astype(int)

# first 3 rows of a subtable
mydholes.table['assay'].loc[(mydholes.table['assay']['BHID']=='0') & (mydholes.table['assay']['FROM']>50), 
                            ['BHID', 'FROM', 'TO', 'Domain1', 'CMPDOM', 'Au']].head()

## Excercise 4: Block models

Create a block model with definition: 
```
xorg = 0
yorg = 0
zorg = 0
dx = 10
dy = 10
dz = 10
nx = 18
ny = 20
nz = 15

```

Calculate the percent or proportion inside domain 1 


In [ ]:
#create an empty model
mymodel = pygslib.blockmodel.Blockmodel(xorg = 0,
                                        yorg = 0,
                                        zorg = 0,
                                        dx = 10,
                                        dy = 10,
                                        dz = 10,
                                        nx = 36/2,
                                        ny = 40/2,
                                        nz = 30/2)

#generate blocks and calculate percent in domain 1
modelvtk = mymodel.fillwireframe(dom1_poly)

In [ ]:
# the model is now created and stored in the property bmtable as pandas dataframe. 
# The property _in is the proportion inside domain 1
mymodel.bmtable.head()

In [ ]:
# the model output is a vtkimageData, you can save it to see in Paraview
pygslib.vtktools.SaveImageData(modelvtk, 'bmodel') # also mymodel.blocks2vtkImageData
print(modelvtk)

In [ ]:
# lets keep only blocks that touch the cell by removing blocks with zero proportion
mymodel.set_blocks (mymodel.bmtable[mymodel.bmtable['__in']>0])

# and save the model as vtkunestructured grid
mymodel.blocks2vtkUnstructuredGrid('bmodel') # this will have extension vtu

There is a problem with the resolution, we need smaller blocks

<img src = 'figures/fig6.JPG' width = '50%' >

Go back and use 5m blocks and twice the number of blocks in each direction to get something like this 

<img src = 'figures/fig7.JPG' width = '50%' >

Hint: in Paraview, right click on the model (pipeline tab) and reload file 

## Excercise 5: Stats and compositing
Now is the time to get ready for interpolation

In this exercise you will do:
- compositing intervals
- declustering 
- statistical analysis 
- variography

### Compositing

It is not required in this case, all the samples are 1m interval. We show you how to do this. Compositing in pygslib is a bit different. 

In [ ]:
# First we remove Au outside domain 1
mydholes.table['assay']['Au_D1'] = mydholes.table['assay']['Au']
mydholes.table['assay'].loc[(mydholes.table['assay']['CMPDOM']!=1) , 'Au_D1'] = None

# see results
# first n rows of a table
mydholes.table['assay'].loc[(mydholes.table['assay']['BHID']=='0') & (mydholes.table['assay']['FROM']>50), 
                            ['BHID', 'FROM', 'TO', 'Domain1', 'CMPDOM', 'Au','Au_D1']].head()

In [ ]:
# then you composite
mydholes.downh_composite(table_name='assay', 
                         variable_name = 'Au_D1', 
                         new_table_name = 'cmp_D1', 
                         cint = 2.7,                  # composite length
                         minlen=-1,                   # minlen will be set cint/2 if <0
                         overwrite =True)

mydholes.table['cmp_D1'].tail()

### Statistical analysis
#### Declustering

In [ ]:
#declustering parameters 
parameters_declus = { 
        'x'      :  mydholes.table['assay'].loc[mydholes.table['assay']['CMPDOM']==1, 'xm'], 
        'y'      :  mydholes.table['assay'].loc[mydholes.table['assay']['CMPDOM']==1, 'ym'],  
        'z'      :  mydholes.table['assay'].loc[mydholes.table['assay']['CMPDOM']==1, 'zm'], 
        'vr'     :  mydholes.table['assay'].loc[mydholes.table['assay']['CMPDOM']==1, 'Au'],   
        'anisy'  :  1.,       
        'anisz'  :  .05,              
        'minmax' :  0,                 
        'ncell'  :  100,                  
        'cmin'   :  10., 
        'cmax'   :  100.,                 
        'noff'   :  8,                    
        'maxcel' :  -1}               

# declustering 
wtopt,vrop,wtmin,wtmax,error, \
xinc,yinc,zinc,rxcs,rycs,rzcs,rvrcr = pygslib.gslib.declus(parameters_declus)

#Plotting declustering optimization results
plt.plot (rxcs, rvrcr, '-o')
plt.xlabel('X cell size')
plt.ylabel('declustered mean')
plt.show()
plt.plot (rycs, rvrcr, '-o')
plt.xlabel('Y cell size')
plt.ylabel('declustered mean')
plt.show()
plt.plot (rzcs, rvrcr, '-o')
plt.xlabel('Z cell size')
plt.ylabel('declustered mean')
plt.show()

Now we fix the cell size to 60 x 60 x 3 

In [ ]:
#declustering parameters 
parameters_declus = { 
        'x'      :  mydholes.table['assay'].loc[mydholes.table['assay']['CMPDOM']==1, 'xm'], 
        'y'      :  mydholes.table['assay'].loc[mydholes.table['assay']['CMPDOM']==1, 'ym'],  
        'z'      :  mydholes.table['assay'].loc[mydholes.table['assay']['CMPDOM']==1, 'zm'], 
        'vr'     :  mydholes.table['assay'].loc[mydholes.table['assay']['CMPDOM']==1, 'Au'],   
        'anisy'  :  1.,       
        'anisz'  :  .05,              
        'minmax' :  0,                 
        'ncell'  :  1,                  
        'cmin'   :  60., 
        'cmax'   :  60.,                 
        'noff'   :  8,                    
        'maxcel' :  -1} 
 

# declustering 
wtopt,vrop,wtmin,wtmax,error, \
xinc,yinc,zinc,rxcs,rycs,rzcs,rvrcr = pygslib.gslib.declus(parameters_declus)

# Adding declustering weight to a drillhole interval table
mydholes.table['assay']['declustwt'] = 1
mydholes.table['assay'].loc[mydholes.table['assay']['CMPDOM']==1, 'declustwt'] = wtopt

# calculating declustered mean
decl_mean = rvrcr[0]

print ('declustered mean:', decl_mean)

In [ ]:
 mydholes.table['assay'].loc[mydholes.table['assay']['CMPDOM']==1, ['BHID','FROM','TO','Au','declustwt']].head()

### Statistical analysis

#### Plots... 

In [ ]:
# parameters to plot clustered cdf 
parameters_probplt = {
    # gslib parameters for histogram calculation  
    'iwt'  : 0, # input boolean (Optional: set True). Use weight variable?
    'va'   : mydholes.table['assay'].loc[mydholes.table['assay']['CMPDOM']==1, 'Au'], # input rank-1 array('d') with bounds (nd). Variable
    'wt'   : mydholes.table['assay'].loc[mydholes.table['assay']['CMPDOM']==1, 'declustwt'], # input rank-1 array('d') with bounds (nd) (Optional, set to array of ones). Declustering weight. 
    # visual parameters for figure (if a new figure is created)
    'figure' : None, # a bokeh figure object (Optional: new figure created if None). Set none or undefined if creating a new figure. 
    'title'  : 'Prob blot', # string (Optional, "Histogram"). Figure title
    'xlabel' : 'Au', # string (Optional, default "Z"). X axis label 
    'ylabel' : 'P[Z<c]', # string (Optional, default "f(%)"). Y axis label
    'xlog' : 1, # boolean (Optional, default True). If true plot X axis in log sale.
    'ylog' : 1, # boolean (Optional, default True). If true plot Y axis in log sale.            
    # visual parameter for the probplt
    'style' : 'cross', # string with valid bokeh chart type 
    'color' : 'blue', # string with valid CSS colour (https://www.w3schools.com/colors/colors_names.asp), or an RGB(A) hex value, or tuple of integers (r,g,b), or tuple of (r,g,b,a) (Optional, default "navy")
    'legend': 'Au non-declustered', # string (Optional, default "NA"). 
    'alpha' : 1, # float [0-1] (Optional, default 0.5). Transparency of the fill colour 
    'lwidth': 0, # float (Optional, default 1). Line width
    # leyend
    'legendloc': 'bottom_right'} #  float (Optional, default 'top_right'). Any of top_left, top_center, top_right, center_right, bottom_right, bottom_center, bottom_left, center_left or center

# parameters to plot declustered cdf 
parameters_probplt_dcl = parameters_probplt.copy()  # make a copy!!!!
parameters_probplt_dcl['iwt']=1                     # and update values
parameters_probplt_dcl['legend']='Au declustered'
parameters_probplt_dcl['color'] = 'red'

# plot clustered and save plot in fig
results_clustered, fig = pygslib.plothtml.probplt(parameters_probplt)


# to plot one on top of the other we add figure to parameter file and plot declustered
parameters_probplt_dcl['figure']= fig
results_declustered, fig = pygslib.plothtml.probplt(parameters_probplt_dcl)

# show the plot
pygslib.plothtml.show(fig)

In [ ]:
# we can also extract the values of the plots: 
print ('stats Declustered')
print ('======================================')
print ('CV',    results_declustered['xcvr'])
print ('Mean', results_declustered['xmen'])
print ('Min', results_declustered['xmin'])
print ('Max', results_declustered['xmax'])
print ('')
print ('stats Clustered')
print ('======================================')
print ('CV',    results_clustered['xcvr'])
print ('Mean', results_clustered['xmen'])
print ('Min', results_clustered['xmin'])
print ('Max', results_clustered['xmax'])

In [ ]:
mydholes.table['assay'].loc[mydholes.table['assay']['CMPDOM']==1, 'Au'].describe()

### Statistical analysis

#### Variography

For now use this variogram model

# In Pygslib
```
vario_model = {
            # Variogram parameters Pygslib
            # ----------
            'c0'         : 0.1,   
            'it'         : [1],    # 
            'cc'         : [.9],     
            'aa'         : [100],   
            'aa1'        : [100],  
            'aa2'        : [20],   
            'ang1'       : [0],   
            'ang2'       : [0],  
            'ang3'       : [-15]}  

```


In gslib 

```
1    0.1                      -nst, nugget effect
1    0.9  0.0   0.0  -15.0     -it,cc,ang1,ang2,ang3
       100.0  100.0  20.0     -a_hmax, a_hmin, a_vert


nst and c0: the number of variogram structures and the nugget
it:  the type of structure
cc:  the c parameter "sill"
ang1,ang2,ang3: the angles defining the geometric anisotropy
aa: also aa_hmax, the maximum horizontal range
aa1: also aa_hmin, the minimum horizontal range
aa2: also aa_vert, the vertical range


it is 

1. Spherical (use actual range)
2. Exponential (use practical range)
3. Gaussian (use practical range)
4. Power law variogram
5. Cosine hole effect model


```

## Excercise 6 Interpolation and validation

First estimate in a single block and review results, then estimate in the entire model

For estimation you may use the function ``pygslib.gslib.kt3d``, which is the GSLIB’s KT3D program modified and embedded into python. KT3D now includes a maximum number of samples per drillhole in the search ellipsoid and the estimation is only in the blocks provided as arrays. 

The input parameters of ``pygslib.gslib.kt3d`` are defined in a large and complicated dictionary. You can get this dictionary by typing 

```
print pygslib.gslib.kt3d.__doc__
```

Note that some parameters are optional. PyGSLIB will initialize those parameters to zero or to array of zeros, for example if you exclude the coordinate Z, PyGSLIB will create an array of zeros in its place.

To understand GSLIB’s KT3D parameters you may read the [GSLIB user manual](https://www.amazon.ca/GSLIB-Geostatistical-Software-Library-Users/dp/0195100158) or [the kt3d gslib program parameter documentation](http://www.statios.com/help/kt3d.html). 

Note that in PyGSLIB the parameters nx, ny and nz are only used by superblock search algorithm, if these parameters are arbitrary the output will be correct, but the running time may be longer.

### Defining the parameter file

In [ ]:
# creating BHID of type integer, this is to be able to use drillhole id in Fortran!
mydholes.txt2intID('assay')
mydholes.table["assay"][['BHID','FROM','TO','BHIDint']].head()

In [ ]:
# creating parameter dictionary for estimation in one block
kt3d_Parameters = {
            # Input Data (Only using intervals in the mineralized domain)
            # ----------
            'x' : mydholes.table["assay"]['xm'][mydholes.table["assay"]['CMPDOM']==1].values, 
            'y' : mydholes.table["assay"]['ym'][mydholes.table["assay"]['CMPDOM']==1].values,
            'z' : mydholes.table["assay"]['zm'][mydholes.table["assay"]['CMPDOM']==1].values,
            'vr' : mydholes.table["assay"]['Au'][mydholes.table["assay"]['CMPDOM']==1].values,
            'bhid' : mydholes.table["assay"]['BHIDint'][mydholes.table["assay"]['CMPDOM']==1].values, # an interger BHID
            # Output (Target) 
            # ----------
            'nx' : 100,  # these parameters are only used to define supperblock search
            'ny' : 100,  
            'nz' : 100, 
            'xmn' : 0,  
            'ymn' : 0,  
            'zmn' : 0,  
            'xsiz' : 5,  
            'ysiz' : 5,   
            'zsiz' : 5, 
            'nxdis' : 5,  
            'nydis' : 5,  
            'nzdis' : 3,  
            'outx' : mymodel.bmtable['XC'][mymodel.bmtable['IJK']==16682].values,  # filter to estimate only on block with IJK 1149229
            'outy' : mymodel.bmtable['YC'][mymodel.bmtable['IJK']==16682].values,
            'outz' : mymodel.bmtable['ZC'][mymodel.bmtable['IJK']==16682].values,
            # Search parameters 
            # ----------
            'radius'     : 60,   
            'radius1'    : 60,   
            'radius2'    : 8,   
            'sang1'      : 0,  
            'sang2'      : 0,   
            'sang3'      : -15,   
            'ndmax'      : 20,    
            'ndmin'      : 7,  
            'noct'       : 0,
            'nbhid'      : 5,   
            # Kriging parameters and options 
            # ----------
            'ktype'      : 1,   # 1 Ordinary kriging 
            'idbg'       : 1,   # 0 no debug 
            # Variogram parameters Pygslib
            # ----------
            'c0'         : 0.1,   
            'it'         : [1],    
            'cc'         : [.9],     
            'aa'         : [100],   
            'aa1'        : [100],  
            'aa2'        : [20],   
            'ang1'       : [0],   
            'ang2'       : [0],  
            'ang3'       : [-15]}   

### Testing the parameters in one block

Only the block with index *IJK* equal to 1149229 was used this time and ``'idbg'`` was set to one in order to get a full output of the last (and unique) block estimate, including the samples selected, kriging weight and the search ellipsoid. 

In [ ]:
# estimating in one block
estimate, debug, summary = pygslib.gslib.kt3d(kt3d_Parameters)

You can export the results to Paraview to better observe the results of the estimate in a single block 

In [ ]:
#saving debug to a csv file using Pandas
pd.DataFrame({'x':debug['dbgxdat'],'y':debug['dbgydat'],'z':debug['dbgzdat'],'wt':debug['dbgwt']}).to_csv('dbg_data.csv', index=False)

# save the search ellipse to a VTK file
pygslib.vtktools.SavePolydata(debug['ellipsoid'], 'search_ellipsoid')

It may look like this

<img src = 'figures/fig8.JPG' widht = '40%' heigth = '40%'>

In [ ]:
# calculate block variance, wee need it for global change of support validation
# you can also calculate this with the function pygslib.gslib.block_covariance(...)
cbb=debug['cbb']

### Estimate in the entire block model
After testing the estimation parameters in few blocks you may be ready to estimate in all the blocks within the mineralized domain. Just update the parameter file to remove the debug option and reassign the target coordinates as the actual blocks coordinate arrays.

In [ ]:
# update parameter file
kt3d_Parameters['idbg'] = 0 # set the debug of
kt3d_Parameters['outx'] = mymodel.bmtable['XC'].values  # use all the blocks 
kt3d_Parameters['outy'] = mymodel.bmtable['YC'].values
kt3d_Parameters['outz'] = mymodel.bmtable['ZC'].values

In [ ]:
# estimating in all blocks
estimate, debug, summary = pygslib.gslib.kt3d(kt3d_Parameters)

In [ ]:
# adding the estimate into the model
mymodel.bmtable['Au_OK'] = estimate['outest']
mymodel.bmtable['Au_ID2'] = estimate['outidpower']
mymodel.bmtable['Au_NN'] = estimate['outnn']
mymodel.bmtable['Au_Lagrange'] = estimate['outlagrange']
mymodel.bmtable['Au_KVar']= estimate['outkvar']

In [ ]:
# exporting block model to VTK (unstructured grid) 
mymodel.blocks2vtkUnstructuredGrid(path='model.vtu')

# exporting to csv using Pandas
mymodel.bmtable['Domain']= 1
mymodel.bmtable[mymodel.bmtable['Au_OK'].notnull()].to_csv('model.csv', index = False)

## Exercise 7: Validations

Basic validations are:

 - visual validation
 - comparison of mean grades
 - swath plots 
 - global change of support (GCOS)


### Visual validations 
Open the model and drillholes in Paraview and inspect: 

- blocks non-estimated
- reproduction of trends and estimation artifacts
- similarity between interpolated grade and nearby composites/assays

This is an example

<img src = 'figures/fig9.JPG' width = '70%' > 

### Mean comparison

In [ ]:
print ("Mean in model OK   :",  mymodel.bmtable['Au_OK'].mean())
print ("Mean in model ID2   :",  mymodel.bmtable['Au_ID2'].mean())
print ("Mean in model NN   :",  mymodel.bmtable['Au_NN'].mean())
print ("Mean in data    :", mydholes.table["assay"]['Au'][mydholes.table["assay"]['CMPDOM']==1].mean())
print ("Declustered mean:", decl_mean)

### Create swath plots

There are two ways of doing swath plots

-	Slicing block model and data and comparing the declustered means of each slice
-	Calculating nearest neighbor in blocks (this is equivalent to declustered values) and comparing means of nearest neighbor estimates with means of other estimation methods along row, columns and levels.  

We do not have a function in pygslib to do that, but we can implement the second option with one line of ``pandas``


In [ ]:
mymodel.bmtable.groupby('XC')[['Au_OK','Au_ID2','Au_NN']].mean().plot()

In [ ]:
mymodel.bmtable.groupby('YC')[['Au_OK','Au_ID2','Au_NN']].mean().plot()

In [ ]:
mymodel.bmtable.groupby('ZC')[['Au_OK','Au_ID2','Au_NN']].mean().plot()

### Global change of support
The process is: 
a) fit point anamorphosis function
b) calculate the support correction coefficient r for the block size you have in your model
c) fit block anamorphosis
d) use the anamorphosis to calculate grade tonnage and compare it with grade tonnage generated with the block model

You have to fit the anamorphosis function, this is easy, normally no teak is required...  

In [ ]:
# Fit anamorphosis by changing, zmax, zmin, and extrapolation function
PCI, H, raw, zana, gauss, z, P, raw_var, PCI_var, fig1 = pygslib.nonlinear.anamor(
                         z = mydholes.table["assay"].loc[mydholes.table['assay']['CMPDOM']==1, 'Au'], 
                         w = mydholes.table["assay"].loc[mydholes.table['assay']['CMPDOM']==1, 'declustwt'], 
                         zmin = mydholes.table["assay"].loc[mydholes.table['assay']['CMPDOM']==1, 'Au'].min(), 
                         zmax = mydholes.table["assay"].loc[mydholes.table['assay']['CMPDOM']==1, 'Au'].max(),
                         zpmin = None, zpmax = None,
                         ymin=-5, ymax=5,
                         ndisc = 5000,
                         ltail=1, utail=4, ltpar=1, utpar=1.5, K=40)

In [ ]:
# calculate the support correction coefficient r
r = pygslib.nonlinear.get_r(Var_Zv = cbb, PCI = PCI)

print ('cbb :', cbb)
print ('r   :', r)

In [ ]:
# fit block anamorphosis
ZV, PV, fig2 = pygslib.nonlinear.anamor_blk( PCI, H, r = r, gauss = gauss, Z = z,
                  ltail=1, utail=1, ltpar=1, utpar=1,
                  raw=raw, zana=zana)

In [ ]:
# calculate grade tonnage courve

cutoff = np.arange(0, 3, 0.01)
tt = []
gg = []
label = []

# calculate GTC from gaussian in block support 
t,ga,gb = pygslib.nonlinear.gtcurve (cutoff = cutoff, z=ZV, p=PV, varred = 1, ivtyp = 0, zmin = 0, zmax = None,
             ltail = 1, ltpar = 1, middle = 1, mpar = 1, utail = 1, utpar = 1,maxdis = 1000)
tt.append(t)
gg.append(ga)
label.append('DGM with block support')

In [ ]:
# see how it look with GCOS. This is also know as global estimation
fig = pygslib.nonlinear.plotgt(cutoff = cutoff, t = tt, g = gg, label = label)

In [ ]:
# to compare global resources with the one estimated we calculate the CDF of the blocks

# cdf of kriging estimate
parameters_probplt = {
        'iwt'  : 0,                             #int, 1 use declustering weight
        'va'   : mymodel.bmtable['Au_OK'][mymodel.bmtable['Au_OK'].notnull()].values,             # array('d') with bounds (nd)
        'wt'   : np.ones(mymodel.bmtable['Au_OK'][mymodel.bmtable['Au_OK'].notnull()].shape[0])} # array('d') with bounds (nd), wight variable (obtained with declust?)


binval_ok,cl_ok,xpt025,xlqt,xmed,xuqt,xpt975,xmin,xmax, \
xcvr,xmen,xvar,error = pygslib.gslib.__plot.probplt(**parameters_probplt)

# cdf of id2
parameters_probplt = {
        'iwt'  : 0,                             #int, 1 use declustering weight
        'va'   : mymodel.bmtable['Au_ID2'][mymodel.bmtable['Au_OK'].notnull()].values,             # array('d') with bounds (nd)
        'wt'   : np.ones(mymodel.bmtable['Au_OK'][mymodel.bmtable['Au_OK'].notnull()].shape[0])} # array('d') with bounds (nd), wight variable (obtained with declust?)

binval_id2,cl_id2,xpt025,xlqt,xmed,xuqt,xpt975,xmin,xmax, \
xcvr,xmen,xvar,error = pygslib.gslib.__plot.probplt(**parameters_probplt)

In [ ]:
# calculate GTC ok 
t,ga,gb = pygslib.nonlinear.gtcurve (cutoff = cutoff, z=cl_ok, p=binval_ok, varred = 1, ivtyp = 2, zmin = 0, zmax = None,
             ltail = 1, ltpar = 1, middle = 1, mpar = 1, utail = 1, utpar = 1,maxdis = 1000)
tt.append(t)
gg.append(ga)
label.append('Ordinary Kriging')

# calculate GTC in block support
t,ga,gb = pygslib.nonlinear.gtcurve (cutoff = cutoff, z=cl_id2, p=binval_id2, varred = 1, ivtyp = 2, zmin = 0, zmax = None,
             ltail = 1, ltpar = 1, middle = 1, mpar = 1, utail = 1, utpar = 1,maxdis = 1000)
tt.append(t)
gg.append(ga)
label.append('Inverse of the Distance 2)')

In [ ]:
fig = pygslib.nonlinear.plotgt(cutoff = cutoff, t = tt, g = gg, label = label)

In [ ]:
# we can plot differences (relative error in grade)
plt.plot (cutoff, gg[0]-gg[1], label = 'DGM - OK')
plt.plot (cutoff, gg[0]-gg[2], label = 'DGM - ID2')
plt.plot (cutoff, np.zeros(cutoff.shape[0]),'--k', label = 'Zero error')
plt.title('relative error in grade')
plt.xlabel ('cutoff')
plt.ylabel ('relative error')
plt.legend()

In [ ]:
# we can plot differences (relative error in tonnage)
plt.plot (cutoff, tt[0]-tt[1], label = 'DGM - OK')
plt.plot (cutoff, tt[0]-tt[2], label = 'DGM - ID2')
plt.plot (cutoff, np.zeros(cutoff.shape[0]),'--k', label = 'Zero error')
plt.legend()
plt.xlabel ('cutoff')
plt.ylabel ('relative error')
plt.title('relative error in tonnage')